<h2> 0. IMPORT REQUIRED LIBRARIES </h2>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

<h2> 1. SET INITIAL REQUIREMENTS </h2>

In [ ]:
# Set the query term to look for
query = 'monitor'

# Set the URL to scrape from in a variable
base_url = 'https://listado.mercadolibre.com.ve/'+ query

# Create an empty list to store the product info
products = []

# Start variable with page number
page_num = 1

<h2>2. LOOP PAGES AND EXTRACT ALL PRODUCTS </h2>

In [ ]:
# Set a loop to go through the pages
while True:
    # Set the URL for the current page number
    url = f'{base_url}_Desde_{page_num}'

    # Make a GET request to the URL and parse the HTML response
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all product elements on the page
    product_elements = soup.find_all('li', class_='ui-search-layout__item')
    
    # If there are no more products, break out of the loop
    if not product_elements:
        break
    
    for element in product_elements:
        # Extract the relevant information about each product
        title_element = element.find('h2', class_='ui-search-item__title')
        title = title_element.text if title_element else None
        
        price_element = element.find('span', class_='price-tag-fraction')
        price = float(price_element.text.replace('.', '')) if price_element else None
        
        link_element = element.find('a', class_='ui-search-link')
        link = link_element['href'] if link_element else None
        
        # Only add the product to the list if its price is greater than or equal to 50
        if price and price >= 50:
            # Store the product information in a dictionary and add it to the list
            product = {'title': title, 'price': price, 'link': link}
            products.append(product)
    
    # Increment the page number and add a delay before making the next request
    page_num += 50
    # Set time delay to 3 seconds 
    time.sleep(3)

# Create a pandas DataFrame from the list of products
df = pd.DataFrame(products)

<h2>3. SORT AND FILTER EXTRACTED DATA </h2>

In [ ]:
# Display the resulting DataFrame
df

In [ ]:
# Sort the DataFrame by price in ascending order
df = df.sort_values(by='price', ascending=True)

In [ ]:
# Remove the products with prices that are not greater than or equal to 100
df = df[df['price'] >= 100]

In [ ]:
# Remove the products with prices that are greater than or equal to 1000
df = df[df['price'] < 1000]

In [ ]:
# Reset the index of the DataFrame
df = df.reset_index(drop=True)

In [ ]:
# Check the resulting DataFrame
df

<h2>4. EXPORT FINAL DATAFRAME </h2>

In [ ]:
# Save the DataFrame to an Excel file
df.to_excel('products.xlsx', index=False)